# Sea Ice Daily Figures for ACCESS-OM2_01

Author: Andrew Kiss; heavily based on Adele Morrison's "Ice Validation mom025"

In [1]:
%matplotlib inline

from glob import glob
import os,sys
import matplotlib.pyplot as plt
import numpy as np
import netCDF4 as nc
from tqdm import tqdm_notebook
from mpl_toolkits.basemap import Basemap
from calendar import month_abbr
from netCDF4 import Dataset
import cmocean as cm
import string

/home/156/aek156/.local/lib/python3.6/site-packages/cmocean/tools.py:76: MatplotlibDeprecationWarning: The is_string_like function was deprecated in version 2.1.
  if not mpl.cbook.is_string_like(rgbin[0]):


In [2]:
# figdir = 'figures/access-om2-01/Ice_Daily_ACCESS-OM2-01/'
figdir = '/g/data/v45/aek156/figures/access-om2-01/Ice_Daily_ACCESS-OM2-01/'
if not os.path.exists(figdir):
    os.makedirs (figdir)

def savefigure(fname):
    plt.savefig(os.path.join(figdir, fname),dpi=100, bbox_inches="tight")  # comment out to disable saving
    return

In [3]:
# model data paths:
model = 'access-om2-01'
expt = '01deg_jra55v13_ryf8485_spinup6'
DataDir = os.path.join('/g/data3/hh5/tmp/cosima/', model)
expdir = os.path.join(DataDir, expt)
# date = '0004-07-05' # nicest view
# date = '0004-07-12'
# fn01 = '/g/data3/hh5/tmp/cosima/' + model + '/' +  expt + '/output023/ice/OUTPUT/iceh.' + date + '.nc'

# sea ice observation data paths:
obsVersion = 3
if obsVersion==3:
    ObsDir = '/g/data1a/v45/aek156/data/NOAA/G02202_V3'  # from http://nsidc.org/data/G02202
    ObsDirExt = '/g/data1a/v45/aek156/data/NOAA/G02135'  # from http://nsidc.org/data/g02135
else:
# v2
    ObsDir = '/g/data/v45/akm157/data/NSIDC/NOAA_G02202_v2_conc_monthly/'
    ObsDirExt = '/g/data/v45/akm157/data/NSIDC/NOAA_G02135_extent_monthly/' # v2.1

# # length of climatology (may be corrected below):
# n_years = 10

# # path to model sea ice data:
# dataFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.????-??.nc'))  # monthly
# dataFileList.sort()
# dataFileListDaily = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.????-??-??.nc'))  # daily
# dataFileListDaily.sort()

# # update n_years to avoid exceeding available data
# if model=='mom025':
#     filesPerYear = 1
# elif model=='mom01v5':
#     filesPerYear = 4
# elif model=='access-om2-01':
#     filesPerYear = 12
# total_years = len(dataFileList)/filesPerYear  # NB: typically not an integer
# n_years = min(n_years, int(total_years) )
# n_files = n_years*filesPerYear
# firstfile = int(total_years-n_years)*filesPerYear  # start of calendar year

# paths to obs data:
if obsVersion==3:
    obsNHFileList = glob(os.path.join(ObsDir, 'north/monthly/*.nc'))
    obsSHFileList = glob(os.path.join(ObsDir, 'south/monthly/*.nc'))
    obsExtNHFileList = glob(os.path.join(ObsDirExt, 'north/monthly/data/*.csv'))
    obsExtSHFileList = glob(os.path.join(ObsDirExt, 'south/monthly/data/*.csv'))
else:
    obsNHFileList = glob(os.path.join(ObsDir, 'nh/*.nc'))
    obsSHFileList = glob(os.path.join(ObsDir, 'sh/*.nc'))
    obsExtNHFileList = glob(os.path.join(ObsDirExt, 'nh/*.csv'))
    obsExtSHFileList = glob(os.path.join(ObsDirExt, 'sh/*.csv'))

obsNHFileList.sort()
obsSHFileList.sort()
obsExtNHFileList.sort()
obsExtSHFileList.sort()


# get model grid data:
gridFileList = glob(os.path.join(expdir, 'output*/ocean/ocean_grid.nc'))
gridFileList.sort()
ncFile = nc.Dataset(gridFileList[0], mode='r')
xt_ocean = ncFile.variables['xt_ocean'][...]
yt_ocean = ncFile.variables['yt_ocean'][...]
lon_t = ncFile.variables['geolon_t'][...]
lat_t = ncFile.variables['geolat_t'][...]
area_t = ncFile.variables['area_t'][...]
# NLAT_half = int(np.shape(area_t)[0]/2)
ht = ncFile.variables['ht'][...]
land_mask = np.copy(ht)
land_mask[np.where(ht>30)] = 0
land_mask[np.where(ht<=30)] = 1

In [4]:
############################################
####### obs climatology:
# for obs just use years 1988-1997, because concentration not available before then and 
# climate change after then

# NH All:
# CN_obs_NH = [[] for _ in range(12)]  # list keeps months separate
CN_obs_NH = [None]*12
for mo in tqdm_notebook(range(12), leave=False, desc='month'):
    # length of climatology:
    obs_list = [IceFile for IceFile in obsNHFileList if (IceFile.find(str(mo+1).zfill(2)+'_v0')>0 and 
        IceFile.find('nh_f')>0 and float(IceFile[-16:-12])>1987 and \
        float(IceFile[-16:-12])<1998)] 
    n_yrs = len(obs_list)
    for IceFile in tqdm_notebook(obs_list, leave=False, desc='year'):
    #     print('opening '+IceFile)
        ncFile = nc.Dataset(IceFile, mode='r')
        tmp = ncFile.variables['seaice_conc_monthly_cdr'][0,...]
#         if len(CN_obs_NH[mo])==0:
        if CN_obs_NH[mo]==None:
            CN_obs_NH[mo] = tmp
        else:
            CN_obs_NH[mo] = CN_obs_NH[mo] + tmp
    # divide by n_yrs and mask land / arctic pole hole:
    CN_obs_NH[mo] = np.ma.masked_where(CN_obs_NH[mo]<0,CN_obs_NH[mo]) / n_yrs
    CN_obs_NH[mo] = np.ma.masked_where(CN_obs_NH[mo]>2,CN_obs_NH[mo])
obs_lat_NH = ncFile.variables['latitude'][...]
obs_lon_NH = ncFile.variables['longitude'][...]

HBox(children=(IntProgress(value=0, description='month', max=12), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

/g/data3/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.6/site-packages/numpy/ma/core.py:3902: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  check = self.filled(0).__eq__(other)


HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

In [5]:
# SH All:
CN_obs_SH = [None]*12
for mo in tqdm_notebook(range(12), leave=False, desc='month'):
    # length of climatology:
    obs_list = [IceFile for IceFile in obsSHFileList if (IceFile.find(str(mo+1).zfill(2)+'_v0')>0 and 
        IceFile.find('sh_f')>0 and float(IceFile[-16:-12])>1987 and \
        float(IceFile[-16:-12])<1998)] 
    n_yrs = len(obs_list)
    for IceFile in tqdm_notebook(obs_list, leave=False, desc='year'):
    #     print('opening '+IceFile)
        ncFile = nc.Dataset(IceFile, mode='r')
        tmp = ncFile.variables['seaice_conc_monthly_cdr'][0,...]
        if CN_obs_SH[mo]==None:
            CN_obs_SH[mo] = tmp
        else:
            CN_obs_SH[mo] = CN_obs_SH[mo] + tmp
    # divide by n_yrs and mask land / arctic pole hole:
    CN_obs_SH[mo] = np.ma.masked_where(CN_obs_SH[mo]<0,CN_obs_SH[mo]) / n_yrs
    CN_obs_SH[mo] = np.ma.masked_where(CN_obs_SH[mo]>2,CN_obs_SH[mo])
obs_lat_SH = ncFile.variables['latitude'][...]
obs_lon_SH = ncFile.variables['longitude'][...]

HBox(children=(IntProgress(value=0, description='month', max=12), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

/g/data3/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.6/site-packages/numpy/ma/core.py:3902: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  check = self.filled(0).__eq__(other)


HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

...

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

HBox(children=(IntProgress(value=0, description='year', max=10), HTML(value='')))

In [6]:
icehFileList = glob(os.path.join(expdir, 'output*/ice/OUTPUT/iceh.0013-*-*.nc'))
icehFileList.sort()

varnames = ['hi', 'aice']
views = ['Arctic_closeup', 'Arctic', 'Antarctic']

for varname in tqdm_notebook(varnames[0:1], leave=False, desc='variables'):
    for view in tqdm_notebook(views, leave=False, desc='views'):
        for fpath in tqdm_notebook(icehFileList, leave=False, desc='day'):
            fname=os.path.basename(fpath)
            date = fname.split('.')[1]
            month = int(date.split('-')[1])
            outname = view + '_' + varname + '_' + date + '.png'

            if os.path.exists(os.path.join(figdir, outname)):
                print('Skipping ' + outname + ' (file exists)')
            else:
                nc = Dataset(fpath, mode='r')
                var = nc.variables[varname][:][0,:,:]
                nc.close()

                if varname == 'aice':
                    desc = 'Ice concentration'
                    levels = np.arange(0,1.01,.01)
                    ticks = [0,.2,.4,.6,.8,1]
                    ext = 'neither'
                elif varname == 'hi':
                    desc = 'Ice thickness (m)'
                    levels = np.arange(0,6.01,.06)
                    ticks = range(7)
                    ext = 'max'
                else:
                    desc = '***UNKNOWN***'
                    levels = np.arange(0,1.01,.1)
                    ticks = [0,.2,.4,.6,.8,1]
                    ext = 'neither'

                font = {'size':24}
                tick_font=24

                fig = plt.figure(figsize=(30,30))
                if view == 'Arctic':
                    m = Basemap(projection ='npstere',boundinglat=45,lon_0=-10,resolution='l')
                elif view == 'Arctic_closeup':
                    m = Basemap(projection ='npstere',boundinglat=70,lon_0=-10,resolution='l')
                else:
                    m = Basemap(projection ='spstere',boundinglat=-50,lon_0=170,resolution='l')

                # m = Basemap(projection ='ortho',resolution='l',lat_0=90,lon_0=45,
                #            llcrnrx=-m.urcrnrx/2.,llcrnry=-m.urcrnry/2.,urcrnrx=m.urcrnrx/2.,urcrnry=m.urcrnry/2.)
                # #             llcrnrlat=60,urcrnrlat=60,
                # #             llcrnrlon=-45,urcrnrlon=45)
                # #             llcrnrlon=loncorners[0],urcrnrlon=loncorners[2]) ,boundinglat=48
                x,y = m(*(lon_t,lat_t))
                ctr = m.contourf(x,y,var,levels=levels,cmap=cm.cm.ice, extend=ext) #, vmin=0, vmax=1.1)
                ctr.cmap.set_over(color='w', alpha=None)
                if view.startswith('Arctic'):
                    xobs,yobs = m(*(obs_lon_NH,obs_lat_NH))
                    m.contour(xobs,yobs,CN_obs_NH[month-1],[0.3],colors='r')
                else:
                    xobs,yobs = m(*(obs_lon_SH,obs_lat_SH))
                    m.contour(xobs,yobs,CN_obs_SH[month-1],[0.3],colors='r')

                plt.title(desc + ', ' + date,font)
                cbar = m.colorbar(ctr, location = 'bottom', pad = "6%")
                cbar.set_label(desc,size=tick_font)
                cbar.set_ticks(ticks)
                cbar_labels=plt.getp(cbar.ax.axes,'xticklabels')
                plt.setp(cbar_labels,fontsize=tick_font)
                m.drawmapboundary(fill_color='gray') # background color - for non-ocean areas
                # fill continents, set lake color same as ocean color.
                # m.fillcontinents(color='gray',lake_color='gray')
                # draw parallels and meridians.
                # label parallels on right and top
                # meridians on bottom and left
                parallels = np.arange(-80.,81,10.)
                # labels = [left,right,top,bottom]
                m.drawparallels(parallels,color='white')#,labels=[False,True,True,False],size=tick_font)
                meridians = np.arange(0.,351.,20.)
                m.drawmeridians(meridians,labels=[True,False,False,True],size=tick_font,color='white')
                savefigure(outname)
                plt.close()

HBox(children=(IntProgress(value=0, description='variables', max=1), HTML(value='')))

HBox(children=(IntProgress(value=0, description='views', max=3), HTML(value='')))

HBox(children=(IntProgress(value=0, description='day', max=365), HTML(value='')))

Skipping Arctic_closeup_hi_0013-01-01.png (file exists)
Skipping Arctic_closeup_hi_0013-01-02.png (file exists)
Skipping Arctic_closeup_hi_0013-01-03.png (file exists)
Skipping Arctic_closeup_hi_0013-01-04.png (file exists)
Skipping Arctic_closeup_hi_0013-01-05.png (file exists)
Skipping Arctic_closeup_hi_0013-01-06.png (file exists)
Skipping Arctic_closeup_hi_0013-01-07.png (file exists)
Skipping Arctic_closeup_hi_0013-01-08.png (file exists)
Skipping Arctic_closeup_hi_0013-01-09.png (file exists)
Skipping Arctic_closeup_hi_0013-01-10.png (file exists)
Skipping Arctic_closeup_hi_0013-01-11.png (file exists)
Skipping Arctic_closeup_hi_0013-01-12.png (file exists)
Skipping Arctic_closeup_hi_0013-01-13.png (file exists)
Skipping Arctic_closeup_hi_0013-01-14.png (file exists)
Skipping Arctic_closeup_hi_0013-01-15.png (file exists)
Skipping Arctic_closeup_hi_0013-01-16.png (file exists)
Skipping Arctic_closeup_hi_0013-01-17.png (file exists)
Skipping Arctic_closeup_hi_0013-01-18.png (file 

/g/data3/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1708: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  limb = ax.axesPatch
/g/data3/hh5/public/apps/miniconda3/envs/analysis3/lib/python3.6/site-packages/mpl_toolkits/basemap/__init__.py:1711: MatplotlibDeprecationWarning: The axesPatch function was deprecated in version 2.1. Use Axes.patch instead.
  if limb is not ax.axesPatch:


Skipping Arctic_closeup_hi_0013-11-25.png (file exists)
Skipping Arctic_closeup_hi_0013-11-26.png (file exists)
Skipping Arctic_closeup_hi_0013-11-27.png (file exists)
Skipping Arctic_closeup_hi_0013-11-28.png (file exists)
Skipping Arctic_closeup_hi_0013-11-29.png (file exists)
Skipping Arctic_closeup_hi_0013-11-30.png (file exists)
Skipping Arctic_closeup_hi_0013-12-01.png (file exists)
Skipping Arctic_closeup_hi_0013-12-02.png (file exists)
Skipping Arctic_closeup_hi_0013-12-03.png (file exists)
Skipping Arctic_closeup_hi_0013-12-04.png (file exists)
Skipping Arctic_closeup_hi_0013-12-05.png (file exists)
Skipping Arctic_closeup_hi_0013-12-06.png (file exists)
Skipping Arctic_closeup_hi_0013-12-07.png (file exists)
Skipping Arctic_closeup_hi_0013-12-08.png (file exists)
Skipping Arctic_closeup_hi_0013-12-09.png (file exists)
Skipping Arctic_closeup_hi_0013-12-10.png (file exists)
Skipping Arctic_closeup_hi_0013-12-11.png (file exists)
Skipping Arctic_closeup_hi_0013-12-12.png (file 

HBox(children=(IntProgress(value=0, description='day', max=365), HTML(value='')))

Skipping Arctic_hi_0013-01-01.png (file exists)
Skipping Arctic_hi_0013-01-02.png (file exists)
Skipping Arctic_hi_0013-01-03.png (file exists)
Skipping Arctic_hi_0013-01-04.png (file exists)
Skipping Arctic_hi_0013-01-05.png (file exists)
Skipping Arctic_hi_0013-01-06.png (file exists)
Skipping Arctic_hi_0013-01-07.png (file exists)
Skipping Arctic_hi_0013-01-08.png (file exists)
Skipping Arctic_hi_0013-01-09.png (file exists)
Skipping Arctic_hi_0013-01-10.png (file exists)
Skipping Arctic_hi_0013-01-11.png (file exists)
Skipping Arctic_hi_0013-01-12.png (file exists)
Skipping Arctic_hi_0013-01-13.png (file exists)
Skipping Arctic_hi_0013-01-14.png (file exists)
Skipping Arctic_hi_0013-01-15.png (file exists)
Skipping Arctic_hi_0013-01-16.png (file exists)
Skipping Arctic_hi_0013-01-17.png (file exists)
Skipping Arctic_hi_0013-01-18.png (file exists)
Skipping Arctic_hi_0013-01-19.png (file exists)
Skipping Arctic_hi_0013-01-20.png (file exists)
Skipping Arctic_hi_0013-01-21.png (file 

HBox(children=(IntProgress(value=0, description='day', max=365), HTML(value='')))

Skipping Antarctic_hi_0013-01-01.png (file exists)
Skipping Antarctic_hi_0013-01-02.png (file exists)
Skipping Antarctic_hi_0013-01-03.png (file exists)
Skipping Antarctic_hi_0013-01-04.png (file exists)
Skipping Antarctic_hi_0013-01-05.png (file exists)
Skipping Antarctic_hi_0013-01-06.png (file exists)
Skipping Antarctic_hi_0013-01-07.png (file exists)
Skipping Antarctic_hi_0013-01-08.png (file exists)
Skipping Antarctic_hi_0013-01-09.png (file exists)
Skipping Antarctic_hi_0013-01-10.png (file exists)
Skipping Antarctic_hi_0013-01-11.png (file exists)
Skipping Antarctic_hi_0013-01-12.png (file exists)
Skipping Antarctic_hi_0013-01-13.png (file exists)
Skipping Antarctic_hi_0013-01-14.png (file exists)
Skipping Antarctic_hi_0013-01-15.png (file exists)
Skipping Antarctic_hi_0013-01-16.png (file exists)
Skipping Antarctic_hi_0013-01-17.png (file exists)
Skipping Antarctic_hi_0013-01-18.png (file exists)
Skipping Antarctic_hi_0013-01-19.png (file exists)
Skipping Antarctic_hi_0013-01-2